<a href="https://colab.research.google.com/github/PiasTanmoy/Vehicle-Recognition-Using-Smart-Sensors/blob/main/Vehicle_Recognition_Cross_Device_Cross_Weather.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import 

In [ ]:
import pandas as pd
import numpy as np
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, MaxPooling1D
from keras.layers.merge import concatenate
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10,5)
plt.rcParams.update({'font.size': 12})

# Read dataset

In [ ]:
car = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/car-pias.csv')
bus = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/bus-pias.csv')
bike = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/bike-david.csv')
rail = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/NEW RAIL/rail1b.csv')

car_rain = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/car-dave-rain.csv')
car_rain_newark_light = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/car-city-newark-light-rain.csv')
car_clear = pd.read_csv('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Data/car-david-clear.csv')

rail_ag = rail[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]
bus_ag = bus[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]
bike_ag = bike[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]
car_ag = car[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]

car_rain_ag = car_rain[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]
car_rain_newark_light_ag = car_rain_newark_light[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]
car_clear_ag = car_clear[['ax', 'ay', 'az', 'wx', 'wy', 'wz']]

# Run

# Dataset Preprocesing

**Hyperparameter: FL = File length**

In [ ]:
FL = 2800 # minimum of all files, the more the better. FL=20,000 is better than FL=2,000

car_ag_fl = car_ag[:FL] 
rail_ag_fl = rail_ag[:FL]
bus_ag_fl = bus_ag[:FL]
bike_ag_fl = bike_ag[:FL]

# generate class label
c0 = np.full((FL), 0).reshape(FL, 1)
c1 = np.full((FL), 1).reshape(FL, 1)
c2 = np.full((FL), 2).reshape(FL, 1)
c3 = np.full((FL), 3).reshape(FL, 1)

print(car_ag_fl.shape, rail_ag_fl.shape, bus_ag_fl.shape, bike_ag_fl.shape, c0.shape, c1.shape, c2.shape, c3.shape)

(2800, 6) (2800, 6) (2800, 6) (2800, 6) (2800, 1) (2800, 1) (2800, 1) (2800, 1)


**Test Weather**

In [ ]:
FL = 2800 # minimum of all files, the more the better. FL=20,000 is better than FL=2,000

car_clear_fl = car_clear_ag[:FL] 
car_rain_ag_fl = car_rain_ag[:FL]
car_rain_newark_light_ag_fl = car_rain_newark_light_ag[:FL]
car_ag_fl = car_ag[:FL] 

rail_ag_fl = rail_ag[:FL]
bus_ag_fl = bus_ag[:FL]
bike_ag_fl = bike_ag[:FL]

# generate class label
c0 = np.full((FL*4), 0).reshape(FL*4, 1)
c1 = np.full((FL), 1).reshape(FL, 1)
c2 = np.full((FL), 2).reshape(FL, 1)
c3 = np.full((FL), 3).reshape(FL, 1)

print(car_clear_fl.shape, car_rain_ag_fl.shape, car_rain_newark_light_ag_fl.shape, c0.shape)

(2800, 6) (2800, 6) (2800, 6) (11200, 1)


# Way 1: Sliding window segmentation

In [ ]:
def segment_dataset(data_class, seg = 200):
  car_np = data_class.to_numpy()
  car_seg = []
  for i in range(car_np.shape[0] - seg):
    car_seg.append(car_np[i:i+seg])
  car_seg = np.array(car_seg) 
  return car_seg


car_seg = segment_dataset(car_ag_fl)
rail_seg = segment_dataset(rail_ag_fl)
bus_seg = segment_dataset(bus_ag_fl)
bike_seg = segment_dataset(bike_ag_fl)

length = car_seg.shape[0]
# generate class label
c0 = np.full((length), 0)
c1 = np.full((length), 1)
c2 = np.full((length), 2)
c3 = np.full((length), 3)

print(car_seg.shape, c0.shape)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

X = np.concatenate((car_seg, rail_seg, bus_seg, bike_seg), axis=0)

Y = np.concatenate((c0, c1, c2, c3), axis = None)
Y = to_categorical(Y)

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)
trainX.shape, testX.shape, trainY.shape, testY.shape

(2600, 200, 6) (2600,)


((8320, 200, 6), (2080, 200, 6), (8320, 4), (2080, 4))

# Cross Weather

In [ ]:
def segment_dataset(data_class, seg = 200):
  car_np = data_class.to_numpy()
  car_seg = []
  for i in range(car_np.shape[0] - seg):
    car_seg.append(car_np[i:i+seg])
  car_seg = np.array(car_seg) 
  return car_seg

car_rain_seg = segment_dataset(car_rain_ag_fl)
#car_rain_newark_light_seg = segment_dataset(car_rain_newark_light_ag_fl)

car_clear_seg = segment_dataset(car_clear_fl)
#car_seg = segment_dataset(car_ag_fl)

rail_seg = segment_dataset(rail_ag_fl)
bus_seg = segment_dataset(bus_ag_fl)
bike_seg = segment_dataset(bike_ag_fl)

length = car_seg.shape[0]

# generate class label
c0 = np.full((length), 0)
c1 = np.full((length), 1)
c2 = np.full((length), 2)
c3 = np.full((length), 3)

print(car_seg.shape, c1.shape)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

trainX = np.concatenate((car_clear_seg, rail_seg, bus_seg, bike_seg), axis=0)
testX = np.concatenate((car_rain_seg, rail_seg, bus_seg, bike_seg), axis=0)

Y = np.concatenate((c0, c1, c2, c3), axis = None)
trainY = to_categorical(Y)
testY = to_categorical(Y)

trainX.shape, testX.shape, trainY.shape, testY.shape

(2600, 200, 6) (2600,)


((10400, 200, 6), (10400, 200, 6), (10400, 4), (10400, 4))

# Test Weather

In [ ]:
def segment_dataset(data_class, seg = 200):
  car_np = data_class.to_numpy()
  car_seg = []
  for i in range(car_np.shape[0] - seg):
    car_seg.append(car_np[i:i+seg])
  car_seg = np.array(car_seg) 
  return car_seg


car_clear_seg = segment_dataset(car_clear_fl)
car_rain_seg = segment_dataset(car_rain_ag_fl)
car_rain_newark_light_seg = segment_dataset(car_rain_newark_light_ag_fl)

car_seg = segment_dataset(car_ag_fl)
rail_seg = segment_dataset(rail_ag_fl)
bus_seg = segment_dataset(bus_ag_fl)
bike_seg = segment_dataset(bike_ag_fl)

length = car_seg.shape[0]
# generate class label
c0 = np.full((length*4), 0)
c1 = np.full((length), 1)
c2 = np.full((length), 2)
c3 = np.full((length), 3)

print(car_seg.shape, c1.shape)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

X = np.concatenate((car_clear_seg, car_rain_seg, car_rain_newark_light_seg, car_seg, rail_seg, bus_seg, bike_seg), axis=0)

Y = np.concatenate((c0, c1, c2, c3), axis = None)
Y = to_categorical(Y)

trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2, random_state=42)
trainX.shape, testX.shape, trainY.shape, testY.shape

# **Run**

**Hyperparameter:**

In [ ]:
num_classes = 2 # depends on the number of classes
#input_shape = (trainX_np.shape[1], trainX_np.shape[2]) # depends on the dimension of the dataset
input_shape = (trainX.shape[1], trainX.shape[2]) # depends on the dimension of the dataset

print(num_classes, input_shape)

2 (200, 6)


# CNN Model

In [ ]:
def make_model(input_shape):
  #input_layer = keras.layers.Input(input_shape)
  
  input_layer = keras.layers.Input(input_shape)

  conv1 = keras.layers.Conv1D(filters=64, kernel_size=9, padding="same")(input_layer)
  conv1 = keras.layers.BatchNormalization()(conv1)
  conv1 = keras.layers.ReLU()(conv1)
  conv1 = MaxPooling1D(pool_size=(2)) (conv1)

  conv2 = keras.layers.Conv1D(filters=64, kernel_size=9, padding="same")(conv1)
  conv2 = keras.layers.BatchNormalization()(conv2)
  conv2 = keras.layers.ReLU()(conv2)
  conv2 = MaxPooling1D(pool_size=(2)) (conv2)

  conv3 = keras.layers.Conv1D(filters=64, kernel_size=9, padding="same")(conv2)
  conv3 = keras.layers.BatchNormalization()(conv3)
  conv3 = keras.layers.ReLU()(conv3)
  conv3 = MaxPooling1D(pool_size=(2)) (conv3)

  connect1 = concatenate([conv1, conv2, conv3], axis=1)


  gap = keras.layers.GlobalAveragePooling1D()(connect1)

  flat = Flatten() (gap)

  dense1 = Dense(1024, activation='relu') (flat)
  drop1 = Dropout(0.5) (dense1)

  dense2 = Dense(256, activation='relu') (drop1)
  drop2 = Dropout(0.5) (dense2)

  dense3 = Dense(64, activation='relu') (drop2)
  drop3 = Dropout(0.5) (dense3)

  output_layer = keras.layers.Dense(num_classes, activation="softmax")(dense3)

  return keras.models.Model(inputs=input_layer, outputs=output_layer)

#input_shape = (100, 6)
#num_classes = 4
model = make_model(input_shape=input_shape)
print(input_shape)
#keras.utils.plot_model(model, show_shapes=True)

(200, 6)


In [ ]:
keras.utils.plot_model(model, show_shapes=True)

In [ ]:
print(model.summary())

**Hyperparameter**

In [ ]:
epochs = 10 # range: 10-1000 
batch_size = 64 # 1, 2, 4, 8, ...., up to length of dataset

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_win_200.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
)
history = model.fit(
    trainX,
    trainY,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split = 0.2,
    verbose=1,
    shuffle = True
)

Epoch 1/10
182/182 [==============================] - 24s 125ms/step - loss: 0.1988 - accuracy: 0.9313 - val_loss: 0.2047 - val_accuracy: 0.9320 - lr: 0.0010
Epoch 2/10
182/182 [==============================] - 26s 141ms/step - loss: 0.0183 - accuracy: 0.9942 - val_loss: 0.0019 - val_accuracy: 0.9997 - lr: 0.0010
Epoch 3/10
182/182 [==============================] - 21s 113ms/step - loss: 0.0181 - accuracy: 0.9937 - val_loss: 0.0050 - val_accuracy: 0.9973 - lr: 0.0010
Epoch 4/10
182/182 [==============================] - 21s 114ms/step - loss: 0.0132 - accuracy: 0.9959 - val_loss: 2.4390e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 5/10
182/182 [==============================] - 21s 113ms/step - loss: 0.0063 - accuracy: 0.9984 - val_loss: 3.1962e-05 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 6/10
182/182 [==============================] - 21s 113ms/step - loss: 0.0219 - accuracy: 0.9943 - val_loss: 6.9054e-04 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 7/10
182/182 [====================

In [ ]:
#model = keras.models.load_model("best_model.h5")

#test_loss, test_acc = model.evaluate(testX, testY)
test_loss, test_acc = model.evaluate(testX, testY)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

114/114 [==============================] - 2s 16ms/step - loss: 1.1963e-04 - accuracy: 1.0000
Test accuracy 1.0
Test loss 0.00011963181896135211


**Save Model**

In [ ]:
model.save('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Code/model weights/best_model_accuracy_100.h5')

**Load Saved Model**

In [ ]:
from keras.models import load_model
model_loaded = load_model('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Code/model weights/best_model_accuracy_100.h5')
score = model_loaded.evaluate(testX, testY, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

407/407 [==============================] - 6s 15ms/step - loss: 1.0501e-04 - accuracy: 1.0000
Test loss: 0.00010500555072212592
Test accuracy: 1.0


# Cross Weather

In [ ]:
model = make_model(input_shape=input_shape)
epochs = 10 # range: 10-1000 
batch_size = 64 # 1, 2, 4, 8, ...., up to length of dataset
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_cross_weather.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
)
history = model.fit(
    trainX,
    trainY,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split = 0.2,
    verbose=1,
    shuffle = True
)

Epoch 1/10
130/130 [==============================] - 15s 109ms/step - loss: 0.1996 - accuracy: 0.9266 - val_loss: 5.7587 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/10
130/130 [==============================] - 14s 109ms/step - loss: 0.0036 - accuracy: 0.9994 - val_loss: 5.1954 - val_accuracy: 0.1149 - lr: 0.0010
Epoch 3/10
130/130 [==============================] - 14s 109ms/step - loss: 2.6109e-04 - accuracy: 1.0000 - val_loss: 1.8507 - val_accuracy: 0.6130 - lr: 0.0010
Epoch 4/10
130/130 [==============================] - 15s 113ms/step - loss: 1.5369e-04 - accuracy: 1.0000 - val_loss: 1.2871 - val_accuracy: 0.7553 - lr: 0.0010
Epoch 5/10
130/130 [==============================] - 15s 118ms/step - loss: 1.3505e-04 - accuracy: 1.0000 - val_loss: 0.6843 - val_accuracy: 0.8183 - lr: 0.0010
Epoch 6/10
130/130 [==============================] - 14s 107ms/step - loss: 0.0271 - accuracy: 0.9912 - val_loss: 0.4184 - val_accuracy: 0.8620 - lr: 0.0010
Epoch 7/10
130/130 [================

In [ ]:
#test_loss, test_acc = model.evaluate(testX, testY)
test_loss, test_acc = model.evaluate(testX, testY)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

325/325 [==============================] - 5s 14ms/step - loss: 0.1299 - accuracy: 0.9633
Test accuracy 0.9632692337036133
Test loss 0.12989501655101776


In [ ]:
model.save('/content/drive/Shareddrives/Vehicle Recognition - Sensor/Code/model weights/cross_weather_accuracy_96.h5')

# Cross Device

In [ ]:
def segment_dataset(data_class, seg = 200):
  car_np = data_class.to_numpy()
  car_seg = []
  for i in range(car_np.shape[0] - seg):
    car_seg.append(car_np[i:i+seg])
  car_seg = np.array(car_seg) 
  return car_seg

car_seg = segment_dataset(car_ag_fl)

car_clear_seg = segment_dataset(car_clear_fl)

rail_seg = segment_dataset(rail_ag_fl)
bus_seg = segment_dataset(bus_ag_fl)
bike_seg = segment_dataset(bike_ag_fl)

length = car_seg.shape[0]

# generate class label
c0 = np.full((length), 0)
c1 = np.full((length), 1)
c2 = np.full((length), 1)
c3 = np.full((length), 1)

print(car_seg.shape, c1.shape)

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 

trainX = np.concatenate((car_clear_seg[:1000], rail_seg, bus_seg, bike_seg), axis=0)
testX = np.concatenate((car_seg[:1000], rail_seg, bus_seg, bike_seg), axis=0)

Y = np.concatenate((c0[:1000], c1, c2, c3), axis = None)
trainY = to_categorical(Y)
Y = np.concatenate((c0[:1000], c1, c2, c3), axis = None)
testY = to_categorical(Y)

trainX.shape, testX.shape, trainY.shape, testY.shape

(2600, 200, 6) (2600,)


((8800, 200, 6), (8800, 200, 6), (8800, 2), (8800, 2))

In [ ]:
model = make_model(input_shape=input_shape)
epochs = 3 # range: 10-1000 
batch_size = 64 # 1, 2, 4, 8, ...., up to length of dataset
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_model_cross_weather.h5", save_best_only=True, monitor="val_loss"
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=20, min_lr=0.0001
    ),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=50, verbose=1),
]
model.compile(
    optimizer="adam",
    loss=keras.losses.categorical_crossentropy,
    metrics=["accuracy"],
)
history = model.fit(
    trainX,
    trainY,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split = 0.2,
    verbose=1,
    shuffle = True
)

Epoch 1/3
110/110 [==============================] - 13s 106ms/step - loss: 0.0945 - accuracy: 0.9509 - val_loss: 3.2556e-05 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 2/3
110/110 [==============================] - 14s 131ms/step - loss: 1.0548e-04 - accuracy: 1.0000 - val_loss: 1.3430e-06 - val_accuracy: 1.0000 - lr: 0.0010
Epoch 3/3
110/110 [==============================] - 11s 104ms/step - loss: 1.9874e-05 - accuracy: 1.0000 - val_loss: 9.0875e-07 - val_accuracy: 1.0000 - lr: 0.0010


In [ ]:
#test_loss, test_acc = model.evaluate(testX, testY)
test_loss, test_acc = model.evaluate(testX, testY)

print("Test accuracy", test_acc)
print("Test loss", test_loss)

275/275 [==============================] - 4s 14ms/step - loss: 2.9962 - accuracy: 0.9320
Test accuracy 0.9320454597473145
Test loss 2.9962399005889893
